## Tensorflow and ProtocolBuffer

In this talk, I'll try to explain how `tensorflow` internally use `protobuf` as serialization tool and how serialized object works under the hook.

If you are not familiar with `protobuf`, I wrote a simplified version of neural network serialization with `protobuf` over [here](https://medium.com/@dboyliao/簡介-google-protocol-buffer-1dd5fa2e1e69).

Oh, It's written in chinese :P

Let's dive into the source code of `tensorflow`!

### What does `tf.train.Saver.save` do?

```
tf.train.Saver.save(self, sess, save_path, global_step=None, latest_filename=None, meta_graph_suffix='meta', write_meta_graph=True, write_state=True)
```

[source code](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/training/saver.py)

Two required arguements:

1. sess (`tf.Session`): a tensorflow Session object
2. save_path (`str`): the path where to write model checkpoint files, used to generate files below
  - **`(save_path)`-`(global_step)`.meta**
  - **`(save_path)`.data-`(step)`-of-`(total)`**
  - **`(save_path)`-`(global_step)`.index**

- `.meta` file --> protobuf file `MetaGraphDef`
- `.data-????-of-????` and `.index` files --> protobuff file for restoring variables
- In the `saver.save` call, `.data` and `.index` files are created with a call to `sess.run` which will invoke an operation node build by `saver`'s builder (See `Saver.__init__` for detail)
- After that, a checkpoint path will be returned.

![write_tensors](images/saver_write_tensors.png)

### MetaGraph

Basically `save_path` is used to detemine:
1. the path and prefix of saved files
2. updating `checkpoint` file

Once `tf.train.Saver` generate file names according to `save_path` and `global_step`, it moves to export **meta graph**

The source code is as following:

![saver-save](images/saver_save.png)


`meta_graph_filename` is now something like `(save_path)-(global_step).meta`


### `tf.train.Saver.export_meta_graph`

```
tf.train.Saver.export_meta_graph(self, filename=None, collection_list=None, as_text=False, export_scope=None, clear_devices=False)
```

The source code is as following:

![saver-export-meta](images/saver_export_meta_graph.png)

As you can see, it's just a wrapper method for calling `export_meta_graph` function, which is defined in the same source file.

![export-meta](images/export_meta_graph.png)

Again, a wrapped call to `meta_graph.export_scoped_meta_graph`.

Basically, what the source code of `meta_graph.export_scoped_meta_graph` do is to construct an instance of `MetaGraphDef`, a class created by protobuf compiler, and save it to disk with standard `protobuf` API. I list a few `.proto` files below, check it out if you want to see the **definitions** for these protobuff objects.

- [meta_graph.proto](https://github.com/tensorflow/tensorflow/blob/r1.3/tensorflow/core/protobuf/meta_graph.proto)
- [saver.proto](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/protobuf/saver.proto)
- [graph.proto](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/graph.proto)
- [node_def.proto](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/node_def.proto)

There are many more of them, good luck!

So, the call stacks to `export_meta_graph` will looks like this:
```
export_meta_graph
  --> meta_graph.export_scoped_meta_graph
    --> build graph_def from default graph or given graph
       -->create a meta_graph_def by the graph_def
         --> save the meta_graph_def to disk (xxx.meta)
```

Similary, the call stacks to `tf.train.import_meta_graph` will reverse above process as follwoing:
```
import_meta_grph
  --> meta_graph.read_meta_graph_file to reconstruct meta_graph_def protobuf object
    --> meta_graph.import_scoped_meta_graph to reconstruct the graph object
      --> return a saver which can be used to restore the session
```

Why returning a saver? What is it for?

Well, we can only reconstruct the graph from a metagraph and the variables in the graph are not yet initialized.

So the saver here is for initializing (restoring) the value of all the tensors in the graph.

### What does tf.Saver.restore do?

Very simple, run the restore operation for a given session to restore the state of the session.

![saver-restore](images/saver_restore.png)

## Simple Summary for Session Saving/Restoration

1. Saving to Disk:
  - `saver.save`
    - saving tensor values to `.data` and `.index` file
    - saving metagraph to `.meta`
2. Restoration from Disk:
  - restore metagraph, a graph consists of saver and graph
    - `tf.train.import_meta_graph`
  - restore session with restored saver
    - `saver.restore`

## You Graph, Freeze!

As you can see, saving a session will create three files at a time, `.data`, `.index` and `.meta`.

It's convenient for prototyping, experiments..etc, but, as for me, I prefer saving my graph in a more compact format when I just want to *use* the graph (like running the trained graph in mobile device). 

Fortunately, `tensorflow` provides some util functions you can use to **freeze** your graph into a more compat format, with just *one* output file.

### tensorflow.python.framework.graph_util.convert_variables_to_constants

- As what its name suggests, `convert_variables_to_constant` will do two things:
  1. `extract_sub_graph`: extract sub graph from given `GraphDef` using BFS (Breath First Search).
  2. construct a new `GraphDef` object and copy the value of all variables in the input `GraphDef` in given session.

![freeze-convert-const](images/convert_const.png)

The next step is simple, just save returned `GraphDef` object (`SerializeToString`):

![save-graphdef](images/save_graph_def.png)

Then load the graph as following:

![load-graphdef](images/load_graph_def.png)

My demo is about showing you how to serve your graph the **hard** way

You can simply use util scripts provided by `tensorflow`, [freeze_graph.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/freeze_graph.py).

But by now, you should be able to understand what is going on in that script.

# Thanks for Your Attension

## Q&A